In [1]:
import joblib
import pandas as pd
import numpy as np
import warnings
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from typing import Union, List
#import ISLP
import imblearn
from imblearn.over_sampling import RandomOverSampler
import warnings
from collections import Counter

warnings.filterwarnings(action='ignore')

In [2]:
df = pd.read_csv('modified_0420.csv')

In [3]:
#df = p.get_df()
df.shape

(1131682, 91)

In [4]:
new_loan_status = []
for i in range(len(df)) :
    if df['loan_status'][i] == 0 :
        new_loan_status.append(1)
    else :
        new_loan_status.append(0)

In [5]:
df.insert(0, 'new_loan_status', new_loan_status)
df

,new_loan_status,Unnamed: 0,loan_amnt,term,int_rate,installment,sub_grade,emp_length,loan_status,delinq_2yrs,...,tax_liens_log,tot_cur_bal_log,tot_hi_cred_lim_log,total_acc_log,total_bal_ex_mort_log,total_bal_il_log,total_bc_limit_log,total_cu_tl_log,total_il_high_credit_limit_log,total_rev_hi_lim_log
0,1,0,6000.0,36,7.97,187.94,4,2,0,1.0,...,0.0,10.820878,11.272280,3.610918,10.820878,10.757158,9.615872,0.000000,10.942208,10.003378
1,0,1,23200.0,60,24.99,680.82,23,10,1,1.0,...,0.0,13.315176,13.386134,4.110874,12.592905,12.533767,9.918425,1.386294,12.582679,10.317450
2,0,2,16000.0,36,7.07,494.55,1,10,1,0.0,...,0.0,11.522113,12.186584,3.367296,10.390594,10.263641,10.987003,0.000000,10.806490,11.157692
3,1,3,4500.0,36,10.42,146.10,7,5,0,0.0,...,0.0,12.178635,12.263577,2.302585,9.166806,7.919356,9.137877,0.000000,9.390242,9.532496
4,1,4,20000.0,36,9.99,645.25,7,10,0,0.0,...,0.0,11.831590,12.137612,3.713572,11.101704,0.000000,9.305741,0.000000,11.199461,10.229224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131677,1,1131677,26500.0,60,17.27,662.45,14,2,0,0.0,...,0.0,12.346938,12.461776,2.833213,10.434557,0.000000,9.846970,0.000000,9.728241,10.507831
1131678,0,1131678,3600.0,36,19.99,133.78,20,7,1,0.0,...,0.0,11.195760,11.378571,2.302585,9.390493,0.000000,8.779711,0.000000,9.501217,9.278092
1131679,1,1131679,19000.0,36,6.68,583.89,2,4,0,0.0,...,0.0,12.388636,12.548072,3.526361,10.837304,0.000000,9.775711,0.000000,10.681688,10.225607
1131680,1,1131680,9600.0,36,14.33,329.65,10,9,0,0.0,...,0.0,11.884551,11.926941,2.197225,9.513256,0.000000,8.343078,0.000000,9.251002,8.881975


In [6]:
df_X = df.drop(columns=['loan_status', 'new_loan_status', 'Unnamed: 0'])
df_y = df['new_loan_status']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size = 0.2, random_state = 42)

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [8]:
pca = PCA(n_components = 50)
pca.fit(df_X)
X_pca = pca.transform(df_X)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, df_y, test_size = 0.2, random_state = 42)
#X_train, X_test, y_train, y_test = train_test_split(X_pca, df_y, test_size=0.2, random_state=42, stratify = y_train)

In [9]:
ros = RandomOverSampler(random_state=42)
X_resampled_pca, y_resampled_pca = ros.fit_resample(X_train_pca, y_train_pca)

## RidgeClassifier

In [24]:
from sklearn.linear_model import RidgeClassifierCV

In [11]:
clfRidge = RidgeClassifierCV().fit(X_train, y_train)

In [10]:
joblib.dump(clfRidge, 'clfRidge.pkl')

['clfRidge.pkl']

In [11]:
y_pred = clfRidge.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.04      0.07     45821
           1       0.80      0.99      0.89    180516

    accuracy                           0.80    226337
   macro avg       0.69      0.52      0.48    226337
weighted avg       0.76      0.80      0.72    226337



In [12]:
clfRidge_over = RidgeClassifierCV().fit(X_resampled, y_resampled)

In [13]:
y_pred = clfRidge_over.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.32      0.65      0.43     45821
           1       0.88      0.66      0.75    180516

    accuracy                           0.66    226337
   macro avg       0.60      0.65      0.59    226337
weighted avg       0.77      0.66      0.69    226337



In [14]:
joblib.dump(clfRidge_over, 'clfRidge_Over.pkl')

['clfRidge_Over.pkl']

In [25]:
clfRidge_pcaover = RidgeClassifierCV().fit(X_resampled_pca, y_resampled_pca)

In [26]:
y_pred = clfRidge_pcaover.predict(X_test_pca)
print(classification_report(y_test_pca, y_pred))

              precision    recall  f1-score   support

           0       0.33      0.65      0.43     45821
           1       0.88      0.66      0.75    180516

    accuracy                           0.66    226337
   macro avg       0.60      0.65      0.59    226337
weighted avg       0.77      0.66      0.69    226337



## LassoClassifier

In [10]:
from sklearn.linear_model import LogisticRegressionCV

In [42]:
clfLasso = LogisticRegressionCV(penalty='l1', solver = 'saga').fit(X_train, y_train)

In [28]:
y_pred = clfLasso.predict(X_test)
print(classification_report(y_test, y_pred))
#48분 걸려서 recall 0.05!

              precision    recall  f1-score   support

           0       0.80      1.00      0.89    180516
           1       0.00      0.00      0.00     45821

    accuracy                           0.80    226337
   macro avg       0.40      0.50      0.44    226337
weighted avg       0.64      0.80      0.71    226337



In [ ]:
clfLasso_over = LogisticRegressionCV(penalty='l1', solver = 'saga').fit(X_resampled, y_resampled)

In [ ]:
y_pred = clfLasso_over.predict(X_test)
print(classification_report(y_test, y_pred))

In [11]:
clfLasso_pcaover = LogisticRegressionCV(penalty='l1', solver = 'saga').fit(X_resampled_pca, y_resampled_pca)

In [12]:
y_pred = clfLasso_pcaover.predict(X_test_pca)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.31      0.67      0.43     45821
           1       0.88      0.63      0.73    180516

    accuracy                           0.64    226337
   macro avg       0.60      0.65      0.58    226337
weighted avg       0.77      0.64      0.67    226337



## LDA, QDA

In [50]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [51]:
clfLDA = LinearDiscriminantAnalysis().fit(X_train, y_train)

In [52]:
y_pred = clfLDA.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.10      0.17     45821
           1       0.81      0.98      0.89    180516

    accuracy                           0.80    226337
   macro avg       0.67      0.54      0.53    226337
weighted avg       0.75      0.80      0.74    226337



In [53]:
clfLDA_over = LinearDiscriminantAnalysis().fit(X_resampled, y_resampled)

In [54]:
y_pred = clfLDA.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.10      0.17     45821
           1       0.81      0.98      0.89    180516

    accuracy                           0.80    226337
   macro avg       0.67      0.54      0.53    226337
weighted avg       0.75      0.80      0.74    226337



In [27]:
# QDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [56]:
clfQDA = QuadraticDiscriminantAnalysis().fit(X_train, y_train)

In [24]:
y_pred = clfQDA.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.80      0.82    180516
           1       0.32      0.36      0.34     45821

    accuracy                           0.71    226337
   macro avg       0.58      0.58      0.58    226337
weighted avg       0.73      0.71      0.72    226337



In [57]:
clfQDA_over = QuadraticDiscriminantAnalysis().fit(X_resampled, y_resampled)

In [58]:
y_pred = clfQDA_over.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.30      0.54      0.39     45821
           1       0.86      0.69      0.76    180516

    accuracy                           0.66    226337
   macro avg       0.58      0.61      0.58    226337
weighted avg       0.74      0.66      0.69    226337



In [28]:
clfQDA_pcaover = QuadraticDiscriminantAnalysis().fit(X_resampled_pca, y_resampled_pca)

In [30]:
y_pred = clfQDA_pcaover.predict(X_test_pca)
print(classification_report(y_test_pca, y_pred))

              precision    recall  f1-score   support

           0       0.32      0.53      0.40     45821
           1       0.86      0.72      0.78    180516

    accuracy                           0.68    226337
   macro avg       0.59      0.63      0.59    226337
weighted avg       0.75      0.68      0.70    226337



사실 LDA와 QDA는 자료의 정규성을 가정하고 하는 거라 자료가 연속이어야 함! 해당 자료는 그 차원에서 문제가 있기는 합니다. LDA와 QDA의 타당성에 대해서는 이와 같은 관점에서 무리가 있습니다.

## Naive Bayes

In [45]:
from sklearn.naive_bayes import GaussianNB

In [46]:
clfNB = GaussianNB().fit(X_train, y_train)

In [47]:
y_pred = clfNB.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.34      0.42      0.38     45821
           1       0.84      0.79      0.82    180516

    accuracy                           0.72    226337
   macro avg       0.59      0.61      0.60    226337
weighted avg       0.74      0.72      0.73    226337



In [48]:
clfNB_over = GaussianNB().fit(X_resampled, y_resampled)

In [49]:
y_pred = clfNB_over.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.31      0.60      0.41     45821
           1       0.87      0.66      0.75    180516

    accuracy                           0.65    226337
   macro avg       0.59      0.63      0.58    226337
weighted avg       0.75      0.65      0.68    226337



## SVM

from PyML import *

In [9]:
from sklearn import svm
clfSVM = svm.SVC()
clfSVM.fit(X_train, y_train)

In [1]:
y_pred = clfSVM.predict(X_test)
print(classification_report(y_test, y_pred))

NameError: name 'clfSVM' is not defined